In [28]:
import os
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

import albumentations as A

In [ ]:
# Need to set
CONFIG_PATH = './work_dirs/ocrnet/ocrnet.py'
TEST_IMAGES_PATH = '/opt/ml/input/data/mmseg/test'
ITER = 'latest'
WORK_DIR = './work_dirs/ocrnet/'

In [20]:
cfg = Config.fromfile(CONFIG_PATH)
root=TEST_IMAGES_PATH

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.work_dir = WORK_DIR
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{ITER}.pth')

In [21]:
dataset = build_dataset(cfg.data.test)
if len(dataset) != 624:
        raise AssertionError('Test dataset should 624 image. Check your test.json')
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2022-04-30 03:29:34,052 - mmseg - INFO - Loaded 624 images


In [24]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 624/624, 11.5 task/s, elapsed: 54s, ETA:     0s

In [43]:
# sample_submisson.csv 열기
submission = pd.read_csv('sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# set resize
input_size = 512
output_size = 256
transformed = A.Compose([A.Resize(output_size, output_size)])
		

# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    temp_mask = []
    mask = np.array(predict, dtype='uint8')
    mask = transformed(image=mask)
    temp_mask.append(mask['image'])
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{ITER}.csv'), index=False)

print('Done')

/tmp/ipykernel_23167/3052015317.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_23167/3052015317.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_23167/3052015317.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
/tmp/ipykernel_23167/3052015317.py:13: FutureWarning: The frame.append method is deprecated and will be removed fro